Sentiment Analysis - positive or Negative => binary classification problem
IMDb -- Movie reviews dataset
Recurrent NN-- Process sequences of data --NlP
* time series
* Text in sentences
recurrent -- the NN contains loopss - output of a given layer becomes the input to the same layer in the next word in aa sequence of words for text sequence
* loops in RNNs help them to learn relationships among the data in the sequence.
Good -- on its own is a posivtive sentiment
Not good has a negative sentiment,, Not is earlier in the seauence
RNN do consider the relationships among earlier and later data in the sequence. here the words that determine sentiment are adjacent. But whn considering text's meaning there can  be many words to consider and abitary  nunmber of words between them.

Long short- Term Memory (LSTM) -- layer that makes the neural network recurrent.

- Predictive text input
- Sentiment Analysis
- Responding to questions with predicted best answers
- Inter- language translation
- automated video closed captioning -- speech recognition
- speech synthesis

#Loading the IMDb Movie Review Dataset

In [1]:
from tensorflow.keras.datasets import imdb

C:\Users\okang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\okang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\okang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\okang\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

88,000  unique words in a dataset
* you can specifyy the number of unique words to import -- training and testing data
* usually 10,000 most frquent occuring words are used 
* Due to systemmemeory limitations and training on cpu -- GPU,, TPU
-- the more data the longer it takes to train-- it may produce better models

In [2]:
#load_data  replaces any words outside the  10,000 with a placeholder
number_of_words=10000

In [3]:
import tensorflow as tf

In [4]:
import numpy as np

In [5]:
#save the np.load
np_load_old=np.load

In [6]:
#modify the defalut parameters of np.load
# *a is a list , **k is an object -- argc
np.load=lambda *a,**k: np_load_old(*a, allow_pickle=True)

In [7]:
(x_train,y_train),(x_test,y_test)= imdb.load_data(num_words=number_of_words)

In [8]:
#x-test and x_xtrain apear as 1 dimentional -- Numpy array of objects(list of intergers)

In [9]:
x_train.shape

(25000,)

In [10]:
x_test.shape

(25000,)

In [11]:
y_train.shape

(25000,)

In [12]:
y_test.shape

(25000,)

In [13]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

# Data Exploration

The arrays y_train and y_test -1D with 0s(negative) and 1s(postive). the x_train and x_test are list og intergers each represenig one reviews's contents.
Keras models require numeric data -- IMDb data is preprocessed for you

In [14]:
%pprint # toggle better printing so thats elements do not display vertically

Pretty printing has been turned OFF


In [15]:
x_train[123]

[1, 307, 5, 1301, 20, 1026, 2511, 87, 2775, 52, 116, 5, 31, 7, 4, 91, 1220, 102, 13, 28, 110, 11, 6, 137, 13, 115, 219, 141, 35, 221, 956, 54, 13, 16, 11, 2714, 61, 322, 423, 12, 38, 76, 59, 1803, 72, 8, 2, 23, 5, 967, 12, 38, 85, 62, 358, 99]

to view the original text you need to know the word to which each number corresponds to. 
The keras dictionary - provides the dictionary that maps words to their indexes.
EACH WORDS VALUE IS ITS FREQUENCY OCCURING WORD ... FOR Example randing 1 -- frequently occuring in that dataset
-Training and the testing data -- have an offset of3. (1+3) = 4-- most frequently occuring word has a value of 4.
the values 0,1,2 are reserved
adding (0) -- all the traing /testingmusut have the sam dimentions 
 some reviews may nee d to be padded with a 0 and some shorted
 start of the sequnce (1) -- a token used by keras internally for learnig purposes
 
 unknown word(2) not loaded -- load_data uses 2 for words with freq ranks > than the num-words

In [16]:
word_to_index=imdb.get_word_index()

In [17]:
word_to_index['great']

84

In [18]:
#great is the 84th most freq word

In [19]:
#lets 
index_to_word={index: word for (word, index)in word_to_index.items()}

In [20]:
[index_to_word[i] for i in range(1,51)]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'not', 'you', 'are', 'his', 'have', 'he', 'be', 'one', 'all', 'at', 'by', 'an', 'they', 'who', 'so', 'from', 'like', 'her', 'or', 'just', 'about', "it's", 'out', 'has', 'if', 'some', 'there', 'what', 'good', 'more']

# Decoding a Movie Review

- To decode a review
    - i-3 accounts for the frequency ratings offsets in the encoded reviews
    - i values 0-2 = ? otherwise should return the word

In [21]:
' '.join([index_to_word.get(i-3, '?') for i in x_train[123]])

'? beautiful and touching movie rich colors great settings good acting and one of the most charming movies i have seen in a while i never saw such an interesting setting when i was in china my wife liked it so much she asked me to ? on and rate it so other would enjoy too'

In [22]:
y_train[123]

1

# Data Preparation

- Number of words per review varies
- Keras requires all samples to have the same dimensions
- Prepare the data
    - Restrict every review to the same number of words
    - Pad some with 0s, truncate others
    - pad-sequences function - reshapes the samples and returns a 2D array

In [23]:
words_per_review = 200

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
x_train = pad_sequences(x_train, maxlen = words_per_review)

In [26]:
x_train.shape

(25000, 200)

In [27]:
x_train

array([[   5,   25,  100, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

# Data Preparation - Cont.

- Reshape the x_test

In [28]:
x_test=pad_sequences(x_test, maxlen = words_per_review)

In [29]:
x_test.shape

(25000, 200)

## Splitting the test data into validation and test data

    - Split the 25,000 test samples into 20,000 test samples and 5000 validation samples
    - We pass vaalidation samples to the models fit method - validation_data argument
        - use the scikit-learn train_test_split function

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
x_test,x_val,y_test,y_val=train_test_split(x_test,y_test,random_state=11, test_size=0.20)


In [32]:
x_test.shape

(20000, 200)

In [33]:
x_val.shape

(5000, 200)

# Creating the Neural Network

In [34]:
from tensorflow.keras.models import Sequential

In [35]:
rnn = Sequential()

In [36]:
rnn

In [37]:
from tensorflow.keras.layers import Dense, LSTM, Embedding

### Adding an Embedding Layer

- In our convne example we used one-hot encoding - into categorical data - others 0 and one 1
- index values that represent words
- 1, 00000000000000000000000000000000000000000000000000000000000000000000000
    - 0 100000000000000000000000000000000000000000000000000000000000000000
- recall cnn.add(Dense(units=10, activation ='softmax')) - 0,1,2,3,4,5,6,7,8,9 => 0.0003, 0.88...
- Will need 10,000 by 10,000 array to represent all the words
- 100,000,000 array to represent the words
- All would be 0 except 1
- all 88,000 +unique words -- 8billion elements

- Reduce dimensionality - RNN process Text sequences, use the embedding layer
- Each encodes each word in a more compact dense-vector representation
- Help RNN - word relations -> this movie is not good --0 this movie is good -- 1 
    * Predefined word embeddings -- Word2Vec and GloVe
        - can load into nn to save training time
        - used to add basic word relationships to model smaller amounts of training  data (available)
        - Improve model accuracy - by looking at the previous word relations -Transfer Learning

In [38]:
rnn.add(Embedding(input_dim = number_of_words, output_dim = 128, input_length = words_per_review))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


- input_dim === Number of unique words
- output_dim-- six of the word embedding
- input_length== Number of words in each input sample

In [39]:
# Adding an LSTM Layer
rnn.add(LSTM(units = 128, dropout = 0.2, recurrent_dropout=0.2))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


- Units -- number of neuronsin that layer (the more the neurons the more the network can remember)
- length of the sequence - 200 -- and the number of classes to predict -- 2 class - 2-200
- dropout- percentage of neurons that are disabled when processing the layer's input and output

## Adding Dense Output Layer

- We need to reduce the LSTM layer's output value to 1(result should be one) + or - We need the value of 1 for the units arguments.
- `sigmoid` activation function - used for binary classification - reduces arbitrary values into the range 0.0 - 1.0, producing a probability

In [40]:
rnn.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the Model and Displaying the Summary

- 'categorical_crossentropy' -- 'binary_crossentropy' regression - 'mean_squared_error'

In [41]:
rnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [42]:
rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


# Trainging and Evaluating the Model
- For every epoch the RNN model takes longer to train than the CNN (Number of parameters)

In [43]:
rnn.fit(x_train, y_train, epochs = 10, batch_size = 32, validation_data = (x_test, y_test))

Train on 25000 samples, validate on 20000 samples
Epoch 1/10
25000/25000 [==============================] - 304s 12ms/sample - loss: 0.4727 - acc: 0.7800 - val_loss: 0.5144 - val_acc: 0.7409
Epoch 2/10
25000/25000 [==============================] - 303s 12ms/sample - loss: 0.4096 - acc: 0.8154 - val_loss: 0.3663 - val_acc: 0.8493
Epoch 3/10
25000/25000 [==============================] - 318s 13ms/sample - loss: 0.2852 - acc: 0.8847 - val_loss: 0.3682 - val_acc: 0.8510
Epoch 4/10
25000/25000 [==============================] - 317s 13ms/sample - loss: 0.2250 - acc: 0.9126 - val_loss: 0.4149 - val_acc: 0.8381
Epoch 5/10
25000/25000 [==============================] - 308s 12ms/sample - loss: 0.2151 - acc: 0.9140 - val_loss: 0.3768 - val_acc: 0.8619
Epoch 6/10
25000/25000 [==============================] - 307s 12ms/sample - loss: 0.1553 - acc: 0.9416 - val_loss: 0.4036 - val_acc: 0.8584
Epoch 7/10
25000/25000 [==============================] - 312s 12ms/sample - loss: 0.1185 - acc: 0.9557 

In [ ]:
# Tuning Deep Learning Models

 - Some variables that can affect the model performance
    - more or less data to train with
    - more or less data to test with
    - more or less data to validate with
    - having more or fewer layers
    - types of layers you use
    - order of the layers
    
 - Some things that we could tune:
    - Try different amounts of training data - 10,000
    - Different number of words per review

In [44]:
results = rnn.evaluate(x_test, y_test)

20000/20000 [==============================] - 20s 1ms/sample - loss: 0.5032 - acc: 0.8442


In [45]:
results

[0.50315741289258, 0.8442]